## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [17]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [35]:
load_dotenv(override=True)
import os
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

openai = OpenAI(base_url = "https://api.anthropic.com/v1/", api_key = anthropic_api_key)

In [19]:
reader = PdfReader("me/resume-clearance.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [20]:
print(resume)

TYLER Y. KIM 
 
 
Permanent Address: 
4701 Caronia Way 
Fairfax, VA  22030 
tyler.ys.kim@gmail.com 
(571) 733-7534 
Current Address:  
Fairfax, VA, 22030 
 
 
 
Education 
 
University of Texas at Austin, Austin, TX 
Masters of Artificial Intelligence, Expected May 2027 
 
University of Virginia, Charlottesville, VA 
Current GPA: 3.9/4.0 
Bachelor of Science in Computer Science, May 2025 
Minor in Data Science, May 2025 
 
Carnegie Mellon University, Pittsburg, PA 
Principles of Computing, Summer 2020 
 
Trinity Christian School, Fairfax, VA 
Summa Cum Laude, May 2020 
 
Virginia Summer Residential Governor’s School for Math, Science and Technology, Lynchburg, VA  
Cryptography, July 2019 
 
 
 
Work Experience and Internships 
Interned as a Data Scientist at the National-Geospatial Intelligence Agency, Summer 2023, Summer 2024 
● Modernized application with python and docker containerization (June 5th - August 11th) 
o Full-time intern working 40 hours per week 
o Used python, pandas,

In [21]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [22]:
name = "Tyler Kim"

In [23]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [24]:
system_prompt

"You are acting as Tyler Kim. You are answering questions on Tyler Kim's website, particularly questions related to Tyler Kim's career, background, skills and experience. Your responsibility is to represent Tyler Kim for interactions on the website as faithfully as possible. You are given a summary of Tyler Kim's background and resume which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Tyler Kim. I graduated from UVA as a Rodman Scholar with a BS in Computer Science and I am currently a graduate student at UT Austin's MSAI program studying AI. I work for the government and actively working on a research project with a professor from UVA where we use generative AI models to correct for systematic bias in particle accelerators.\nWhen time permits, I enjoy playing video games, exercising, and learning new things in my free ti

In [25]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="claude-sonnet-4-5", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [26]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [27]:
# Create a Pydantic model for the Evaluation
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [28]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [29]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [41]:
import os
# from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
# client = genai.Client()

google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set - please head to the troubleshooting guide in the setup folder")


gemini = OpenAI(
    base_url = "https://api.anthropic.com/v1/", api_key = anthropic_api_key
)

Google API Key exists and begins AIzaSyBt


In [42]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="claude-sonnet-4-5", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [43]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="claude-sonnet-4-5", messages=messages)
reply = response.choices[0].message.content

In [44]:
reply

"No, I don't hold any patents. My work has been focused on research and academic projects, including my current research on using diffusion models as digital twins for physics-based applications and my previous work on explainable AI using swarm optimization algorithms. While these are interesting research areas, none of this work has resulted in patented inventions yet.\n\nMost of my experience has been in academic research settings and internships at the National Geospatial-Intelligence Agency, where the work contributes to broader projects rather than individual patent applications. Is there a particular area of my research or technical work you'd like to know more about?"

In [45]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is an excellent response. The Agent correctly states that Tyler does not hold any patents based on the provided information. The response is appropriately professional, acknowledges the question directly, and provides relevant context about Tyler's research work without being defensive. It also includes a helpful follow-up question to keep the conversation engaging, which aligns with the instruction to be professional and engaging with potential clients or employers. The tone is natural and honest, maintaining Tyler's character while staying factual.")

In [53]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="claude-sonnet-4-5", messages=messages)
    return response.choices[0].message.content

In [54]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="claude-sonnet-4-5", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [55]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The Agent provided no response to the User's question about patent ownership. This is unacceptable - the Agent must always provide a response. Based on the provided context (summary and LinkedIn profile), there is no mention of Tyler Kim owning any patents. The Agent should have responded professionally, clarifying that Tyler does not currently hold any patents but highlighting his research experience and technical projects instead. An empty response fails to engage the user and does not represent Tyler Kim professionally.
